In [ ]:
import pandas as pd
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from scipy.interpolate import interp1d

# --- TASK 1: Price Forecasting & Interpolation ---
def get_price_predictor(csv_path, forecast_months=12):
    df = pd.read_csv(csv_path, parse_dates=['Dates'])
    df.set_index('Dates', inplace=True)
    df = df.sort_index().resample('ME').mean() # 'ME' for Month End
    df['Prices'] = df['Prices'].interpolate()

    # Model fitting
    model = ExponentialSmoothing(df['Prices'], trend='add', seasonal='mul', seasonal_periods=12)
    fit = model.fit()

    # Forecasting future prices
    forecast = fit.forecast(forecast_months).astype(float)
    future_dates = [df.index[-1] + relativedelta(months=i+1) for i in range(forecast_months)]

    # Combine historical and forecast
    df_forecast = pd.DataFrame({'Prices': forecast.values}, index=future_dates)
    df_combined = pd.concat([df, df_forecast]).reset_index().rename(columns={'index': 'Dates'})
    df_combined['Ordinal'] = df_combined['Dates'].map(datetime.toordinal)

    # Creating the continuous price function
    price_interp = interp1d(df_combined['Ordinal'], df_combined['Prices'], kind='linear', fill_value='extrapolate')

    def price_fn(date):
        return float(price_interp(date.toordinal()))

    return price_fn

In [ ]:
# --- TASK 2: Sequential Pricing Model ---
def price_contract(in_dates, out_dates, price_fn, rate, storage_cost_rate, total_vol, injection_withdrawal_cost_rate):
    volume = 0
    buy_cost = 0
    cash_in = 0

    # Ensure all dates are sorted chronologically
    all_dates = sorted(set(in_dates + out_dates))

    for start_date in all_dates:
        # Get the market price for the specific date from our predictor
        current_price = price_fn(start_date)

        # Injection Logic
        if start_date in in_dates:
            if volume <= total_vol - rate:
                volume += rate
                buy_cost += (rate * current_price) + (rate * injection_withdrawal_cost_rate)
                print(f"Injected gas on {start_date.date()} at ${current_price:.2f}")
            else:
                print(f"Injection impossible on {start_date.date()}: Facility Full")

        # Withdrawal Logic
        elif start_date in out_dates:
            if volume >= rate:
                volume -= rate
                cash_in += (rate * current_price) - (rate * injection_withdrawal_cost_rate)
                print(f"Extracted gas on {start_date.date()} at ${current_price:.2f}")
            else:
                print(f"Extraction impossible on {start_date.date()}: Insufficient Gas")

    # Calculate storage cost based on duration
    # Ceiling of months between first injection and last withdrawal
    store_cost = math.ceil((max(out_dates) - min(in_dates)).days / 30) * storage_cost_rate

    return round(cash_in - store_cost - buy_cost, 2)

In [ ]:
# --- EXAMPLE EXECUTION ---
if __name__ == "__main__":
    # Path to your CSV
    predictor = get_price_predictor("Nat_Gas.csv")

    # Define trade schedule
    in_dates = [datetime(2024, 6, 1), datetime(2024, 7, 1)]
    out_dates = [datetime(2024, 12, 1), datetime(2025, 1, 1)]

    # Valuation
    contract_value = price_contract(
        in_dates, out_dates, predictor,
        rate=100000,
        storage_cost_rate=10000,
        total_vol=500000,
        injection_withdrawal_cost_rate=0.0005
    )

    print(f"\nThe total value of the contract is: ${contract_value}")

/tmp/ipython-input-1906036444.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(csv_path, parse_dates=['Dates'])


Injected gas on 2024-06-01 at $11.40
Injected gas on 2024-07-01 at $11.50
Extracted gas on 2024-12-01 at $12.73
Extracted gas on 2025-01-01 at $13.14

The total value of the contract is: $216304.93
